In [16]:
import mimetypes
import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import json

BLOCK_SIZE = 8

def extract_and_chunk(file_path):
    filetype = mimetypes.guess_type(file_path)[0]

    if filetype == 'application/pdf':
        return chunk_pdf(file_path)
    elif filetype == 'text/plain':
        with open(file_path) as f:
            return chunk_text(f.read())
    elif filetype == 'text/csv':
        return chunk_text(pd.read_csv(file_path).to_string(index=False))
    elif filetype == 'application/json':
        return chunk_text(json.dumps(json.load(open(file_path)), indent=2))
    elif filetype and filetype.startswith('image/'):
        return [handle_image(file_path)]
    else:
        raise ValueError("Unsupported file type.")

def chunk_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        pages = [page.extract_text() or "" for page in pdf.pages]
        return ["\n".join(pages[i:i+BLOCK_SIZE]) for i in range(0, len(pages), BLOCK_SIZE)]

def chunk_text(text):
    paras = text.split("\n\n")
    return ["\n\n".join(paras[i:i+BLOCK_SIZE]) for i in range(0, len(paras), BLOCK_SIZE)]

def handle_image(file_path):
    text = pytesseract.image_to_string(Image.open(file_path)).strip()
    if len(text.split()) > 10:
        return text 
    else:
        return f"[IMAGE]{file_path}"


In [17]:
def detect_and_extract_text(file_path):
    filetype = mimetypes.guess_type(file_path)[0]

    if filetype == 'application/pdf':
        return extract_text_from_pdf(file_path)
    elif filetype == 'text/plain':
        return extract_text_from_txt(file_path)
    elif filetype == 'text/csv':
        return extract_text_from_csv(file_path)
    elif filetype == 'application/json':
        return extract_text_from_json(file_path)
    elif filetype and filetype.startswith('image/'):
        return extract_text_from_image(file_path)
    else:
        raise ValueError("Unsupported file type.")

In [18]:
from PIL import Image
from google import genai
import os
import dotenv
import re

dotenv.load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))


def sanitize_sensitive_data(text):
    text = re.sub(r'(?i)(api[_-]?key\s*[:=]\s*)([^\s\'"]+)', r'\1[MASKED API KEY]', text)
    text = re.sub(r'(?i)(secret|password|token)\s*[:=]\s*([^\s\'"]+)', r'\1: [MASKED PASSWORD OR SECRET]', text)
    text = re.sub(r'(?i)(AWS|GCP|Azure)[-_ ]?(key|secret|token)[^\n]*', '[MASKED CLOUD CREDENTIAL]', text)
    return text

def convert_to_markdown_with_gemini(content):
    if content.startswith("[IMAGE]"):
        path = content.replace("[IMAGE]", "").strip()
        image = Image.open(path)
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[
                image,
                """Generate a single-line caption for the image in plain text. 
                Do not include any Markdown formatting or image syntax—only output the caption itself."""
            ]
        )
        caption = response.text.strip()
        print(f"Generated caption for image: {caption}")
        return f"{caption}"
    else:
        safe_content = sanitize_sensitive_data(content)
        prompt = (
            "Respond with only valid Markdown — do not include any explanations, apologies, or introductions. "
            "Only output the Markdown content, and redact any confidential data such as passwords, API keys, tokens, or secrets using [MASKED]. "
            "Preserve proper line breaks and formatting exactly as needed for Markdown to render correctly."
        )
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[safe_content, prompt]
        )
        return response.text.strip()


In [ ]:
import requests
import uuid

def pipeline(file_path):
    try:
        content_blocks = extract_and_chunk(file_path)
        markdown_blocks = [convert_to_markdown_with_gemini(block) for block in content_blocks]
        full_markdown = "\n\n".join(markdown_blocks)

        filetype = mimetypes.guess_type(file_path)[0]
        total_chunks = len(content_blocks)

        metadata = {}

        metadata["source"] = os.path.basename(file_path)
        metadata["filetype"] = filetype
        metadata["total_chunks"] = total_chunks

        response = requests.post(
            "http://localhost:9876/addData",
            json={
                "id": uuid.uuid4().hex,
                "text": full_markdown,
                "meta": metadata
            }
        )
        response.raise_for_status()
        print("Uploaded to server:", response.json())

        with open(f"{os.path.splitext(file_path)[0]}.md", "w") as f:
            f.write(full_markdown)

        return full_markdown
    except Exception as e:
        print(f"[Error] {e}")
        return None


In [22]:
# pipeline("../testData/cat.jpg")
# pipeline("../testData/test-123.pdf")
# pipeline('../testData/emc-2.csv')
# pipeline('.../testData/ss.png')
pipeline('../testData/emc-2.json')
# pipeline(('../testData/env.txt'))

Uploaded to server: {'data': {'id': '084843ad5f814422a92e5cf5134805b6', 'meta': {'filetype': 'application/json', 'source': 'emc-2.json', 'total_chunks': 1}, 'text': '```\n| Event Name            | Team Reg | Team Part | External Reg | External Part | 1st Reg | 1st Part | 2nd Reg | 2nd Part | 2nd Lateral Reg | 2nd Lateral Part | 3rd Reg | 3rd Part | 3rd Lateral Reg | 3rd Lateral Part | 4th Reg | 4th Part | 4th Lateral Reg | 4th Lateral Part |\n|-----------------------|----------|-----------|--------------|---------------|---------|----------|---------|----------|-----------------|-------------------|---------|----------|-----------------|-------------------|---------|----------|-----------------|-------------------|\n| Animeverse            | 45       | 39        | 18           | 15            | 53      | 51       | 29      | 29       | 8               | 6                 | 13      | 11       | 1               | 1                 | 1       | 1       | 1               | 1                

'```\n| Event Name            | Team Reg | Team Part | External Reg | External Part | 1st Reg | 1st Part | 2nd Reg | 2nd Part | 2nd Lateral Reg | 2nd Lateral Part | 3rd Reg | 3rd Part | 3rd Lateral Reg | 3rd Lateral Part | 4th Reg | 4th Part | 4th Lateral Reg | 4th Lateral Part |\n|-----------------------|----------|-----------|--------------|---------------|---------|----------|---------|----------|-----------------|-------------------|---------|----------|-----------------|-------------------|---------|----------|-----------------|-------------------|\n| Animeverse            | 45       | 39        | 18           | 15            | 53      | 51       | 29      | 29       | 8               | 6                 | 13      | 11       | 1               | 1                 | 1       | 1       | 1               | 1                 |\n| Locked in Reality     | 38       | 32        | 21           | 15            | 51      | 51       | 24      | 13       | 1               | 0                 | 1

In [14]:
searchResp = requests.post("http://localhost:9876/searchData", json={
  "text": "sherlocked",
  "n_results": 3
})
print(searchResp.json())

{'results': [{'id': '20695a37eb7d4cdeb68ccc4ea73426dd', 'text': '```markdown\n| Event Name            | Team Reg | Team Part | External Reg | External Part | 1st Reg | 1st Part | 2nd Reg | 2nd Part | 2nd Lateral Reg | 2nd Lateral Part | 3rd Reg | 3rd Part | 3rd Lateral Reg | 3rd Lateral Part | 4th Reg | 4th Part | 4th Lateral Reg | 4th Lateral Part |\n|-----------------------|----------|-----------|--------------|---------------|---------|----------|---------|----------|-----------------|--------------------|---------|----------|-----------------|--------------------|---------|----------|-----------------|--------------------|\n| Animeverse            | 45       | 39        | 18           | 15            | 53      | 51       | 29      | 29       | 8               | 6                  | 13      | 11       | 1               | 1                  | 1       | 1        | 1               | 1                  |\n| Locked in Reality     | 38       | 32        | 21           | 15            | 51